# Chapter 2: Opening and Reading NetCDF files using STAC
Creators: Ann Windnagel and Robyn Marowitz

Affiliation: [National Snow And Ice Data Center](https://nsidc.org/home)

## Learning Objectives

In this notebook we will learn how to open and read NetCDF files using STAC. While we do this we will explore the features and vairables of the STAC Collection and Items.

### Import relevant packages
If you're interested in learning more the documentation for each package is available and can be very informative. 

We will use [pystac](https://pystac.readthedocs.io/en/stable/) to explore our STAC collection and items and their details. [NetCDF4](https://pystac.readthedocs.io/en/stable/) is a Python interface to the netCDF C library. That is used to read in information from NetCDF files. [Xarray](https://docs.xarray.dev/en/stable/) makes working with labelled multi-dimensional arrays, such as NetCDF, in Python simple and efficient.


In [1]:
import xarray as xr
import numpy
import pandas
import packaging
import aiohttp

import fsspec
import pystac
import netCDF4
from netCDF4 import Dataset
import h5netcdf

## STAC Collections

[A STAC Collection](https://github.com/radiantearth/stac-spec/tree/master/collection-spec) object is used to describe a group of reltaed Items.  

To begin we will assign our `collection.json` to a variable `collection` so that we can explore it using `pystac`.

In [2]:
collection = pystac.Collection.from_file('https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/collection.json')

## Explore Collection Information

In the following cells we will look at how the Collection is organized. Using the `describe` method we will see all of the `Items` listed. In the `Item id` there is already a wealth of information - `nh` means northern hemisphere and `sh` means southern hemisphere.

In [3]:
collection.describe()

* <Collection id=noaa-cdr-sea-ice-concentration>
  * <Item id=seaice_conc_daily_nh_1996_v04r00>
  * <Item id=seaice_conc_monthly_nh_197811_202312_v04r00>
  * <Item id=seaice_conc_monthly_sh_197811_202312_v04r00>
  * <Item id=seaice_conc_daily_sh_2023_v04r00>
  * <Item id=seaice_conc_daily_sh_2022_v04r00>
  * <Item id=seaice_conc_daily_sh_2021_v04r00>
  * <Item id=seaice_conc_daily_sh_2020_v04r00>
  * <Item id=seaice_conc_daily_sh_2019_v04r00>
  * <Item id=seaice_conc_daily_sh_2018_v04r00>
  * <Item id=seaice_conc_daily_sh_2017_v04r00>
  * <Item id=seaice_conc_daily_sh_2016_v04r00>
  * <Item id=seaice_conc_daily_sh_2015_v04r00>
  * <Item id=seaice_conc_daily_sh_2014_v04r00>
  * <Item id=seaice_conc_daily_sh_2013_v04r00>
  * <Item id=seaice_conc_daily_sh_2012_v04r00>
  * <Item id=seaice_conc_daily_sh_2011_v04r00>
  * <Item id=seaice_conc_daily_sh_2010_v04r00>
  * <Item id=seaice_conc_daily_sh_2009_v04r00>
  * <Item id=seaice_conc_daily_sh_2008_v04r00>
  * <Item id=seaice_conc_daily_sh_20

In [4]:
print('Title: ',collection.title)
# look into pretty print
print('Description: ',collection.description)

Title:  Sea Ice Concentration CDR
Description:  The Sea Ice Concentration Climate Data Record (CDR) provides a consistent daily and monthly time series of sea ice concentrations for both the north and south Polar Regions on a 25 km x 25 km grid. These data can be used to estimate how much of the ocean surface is covered by ice, and monitor changes in sea ice concentration. The CDR combines concentration estimates using two algorithms developed at the NASA Goddard Space Flight Center (GSFC). Gridded brightness temperatures acquired from a number of Defense Meteorological Satellite Program (DMSP) passive microwave radiometers provide the necessary input to produce the dataset.


## Get all items 

In [5]:
items = list(collection.get_all_items())

print(f"Number of items: {len(items)}")

Number of items: 22


## Explore an item

We will select one item and explore its metadata.

In [6]:
daily_sh_2023_item = collection.get_item("seaice_conc_daily_sh_2023_v04r00", recursive=True)

In [7]:
daily_sh_2023_item.geometry

{'type': 'Polygon',
 'coordinates': [[[180.0, -90.0],
   [180.0, -41.45],
   [-180.0, -41.45],
   [-180.0, -90.0],
   [180.0, -90.0]]]}

In [8]:
daily_sh_2023_item.bbox

[-180.0, -90.0, 180.0, -41.45]

In [9]:
daily_sh_2023_item.properties['start_datetime']


'2023-01-01T00:00:00Z'

In [10]:
daily_sh_2023_item.properties['end_datetime']


'2023-12-31T23:59:59Z'

In [11]:
daily_sh_2023_item.set_datetime(daily_sh_2023_item.properties['start_datetime'])

In [12]:
daily_sh_2023_item.datetime

'2023-01-01T00:00:00Z'

## Read NetCDF

In [13]:
# Access the NetCDF asset and see the components
daily_sh_2023_item.assets['netcdf']

<Asset href=https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc>

In [14]:
netcdf_url = daily_sh_2023_item.assets['netcdf'].href
netcdf_url

'https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc'

In [15]:
fs = fsspec.filesystem('https')
ds = xr.open_dataset(fs.open(netcdf_url))
ds

<xarray.Dataset> Size: 1GB
Dimensions:                      (tdim: 365, y: 332, x: 316)
Coordinates:
    time                         (tdim) datetime64[ns] 3kB ...
    xgrid                        (x) float32 1kB ...
    ygrid                        (y) float32 1kB ...
Dimensions without coordinates: tdim, y, x
Data variables:
    cdr_seaice_conc              (tdim, y, x) float32 153MB ...
    nsidc_bt_seaice_conc         (tdim, y, x) float32 153MB ...
    nsidc_nt_seaice_conc         (tdim, y, x) float32 153MB ...
    projection                   |S1 1B ...
    qa_of_cdr_seaice_conc        (tdim, y, x) float32 153MB ...
    spatial_interpolation_flag   (tdim, y, x) float32 153MB ...
    stdev_of_cdr_seaice_conc     (tdim, y, x) float32 153MB ...
    temporal_interpolation_flag  (tdim, y, x) float32 153MB ...
    latitude                     (y, x) float32 420kB ...
    longitude                    (y, x) float32 420kB ...
Attributes: (12/39)
    Conventions:               CF-1.6, ACDD-1.3
    title:                     NOAA/NSIDC Climate Data Record of Passive Micr...
    references:                Comiso, J. C., and F. Nishio. 2008. Trends in ...
    comment:                   none
    program:                   NOAA Climate Data Record Program
    cdr_variable:              cdr_seaice_conc
    ...                        ...
    acknowledgment:            This project was supported in part by a grant ...
    platform:                  DMSP 5D-3/F17 > Defense Meteorological Satelli...
    sensor:                    SSMI/S > Special Sensor Microwave Imager/Sounder
    date_created:              2023-09-26T20:17:19Z
    time_coverage_start:       2023-01-01T00:00:00Z
    time_coverage_end:         2023-12-31T23:59:59Z

In [16]:
# Pull out sea ice concentration variable